In [40]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
import time
import csv

def setup_driver():
    service = Service(ChromeDriverManager().install())
    driver = webdriver.Chrome(service=service)
    return driver

def extract_quotes(driver):
    driver.get("https://quotes.toscrape.com/js-delayed/")
    quotes = []

    while True:
        WebDriverWait(driver, 10).until(
            EC.presence_of_element_located((By.CLASS_NAME, 'quote'))
        )

        # Extrai as citações da página atual
        quote_elements = driver.find_elements(By.CLASS_NAME, 'quote')
        for quote in quote_elements:
            text = quote.find_element(By.CLASS_NAME, 'text').text
            author = quote.find_element(By.CLASS_NAME, 'author').text
            tags = [tag.text for tag in quote.find_elements(By.CLASS_NAME, 'tag')]
            quotes.append({'Citação': text, 'Autor': author, 'Tags': ', '.join(tags)})

        # Verifica se há uma próxima página
        try:
            next_button = driver.find_element(By.CSS_SELECTOR, 'li.next > a')
            next_button.click()
            time.sleep(2)  # Espera o carregamento da próxima página
        except:
            # Se não houver mais páginas, encerra o loop
            break

    return quotes

def save_to_csv(quotes, filename='quotes.csv'):
    with open(filename, mode='w', newline='', encoding='utf-8') as file:
        writer = csv.DictWriter(file, fieldnames=['Citação', 'Autor', 'Tags'])
        writer.writeheader()
        writer.writerows(quotes)

def main():
    driver = setup_driver()
    try:
        quotes = extract_quotes(driver)
        save_to_csv(quotes)
        print(f"Dados extraídos e salvos com sucesso! Total de citações: {len(quotes)}")
    finally:
        driver.quit()

if __name__ == "__main__":
    main()

import pandas as pd

# Lê o arquivo CSV
try:
    df = pd.read_csv('quotes.csv')
except FileNotFoundError:
    print("Arquivo 'quotes.csv' não encontrado. Execute a Parte 1 primeiro.")
    exit()

# Exibe o número de citações
num_quotes = df.shape[0]
print(f"Número de citações: {num_quotes}")

# Identifica o autor mais recorrente
most_frequent_author = df['Autor'].mode()[0]
print(f"Autor mais recorrente: {most_frequent_author}")

# Identifica a tag mais utilizada
all_tags = df['Tags'].str.split(', ', expand=True).stack()
most_frequent_tag = all_tags.mode()[0]
print(f"Tag mais utilizada: {most_frequent_tag}")

# Exibe todas as citações, autores e tags
print("\nTodas as citações:")
for index, row in df.iterrows():
    print(f"\nCitação {index + 1}:")
    print(f"  Citação: {row['Citação']}")
    print(f"  Autor: {row['Autor']}")
    print(f"  Tags: {row['Tags']}")

# Exibe todos os autores únicos
print("\nTodos os autores únicos:")
unique_authors = df['Autor'].unique()
for author in unique_authors:
    print(f"  - {author}")

# Exibe todas as tags únicas
print("\nTodas as tags únicas:")
all_tags = df['Tags'].str.split(', ', expand=True).stack().unique()
for tag in all_tags:
    print(f"  - {tag}")


Dados extraídos e salvos com sucesso! Total de citações: 100
Número de citações: 100
Autor mais recorrente: Albert Einstein
Tag mais utilizada: love

Todas as citações:

Citação 1:
  Citação: “The world as we have created it is a process of our thinking. It cannot be changed without changing our thinking.”
  Autor: Albert Einstein
  Tags: change, deep-thoughts, thinking, world

Citação 2:
  Citação: “It is our choices, Harry, that show what we truly are, far more than our abilities.”
  Autor: J.K. Rowling
  Tags: abilities, choices

Citação 3:
  Citação: “There are only two ways to live your life. One is as though nothing is a miracle. The other is as though everything is a miracle.”
  Autor: Albert Einstein
  Tags: inspirational, life, live, miracle, miracles

Citação 4:
  Citação: “The person, be it gentleman or lady, who has not pleasure in a good novel, must be intolerably stupid.”
  Autor: Jane Austen
  Tags: aliteracy, books, classic, humor

Citação 5:
  Citação: “Imperfection is

In [41]:
import smtplib

# Cria uma instância do servidor SMTP
server = smtplib.SMTP('smtp.gmail.com', 587)  # Use a porta 587 para TLS
server.starttls()  # Inicia a conexão TLS
server.login('login', 'senha')  # Faça login com suas credenciais


(235, b'2.7.0 Accepted')

In [44]:
import smtplib
from email.mime.multipart import MIMEMultipart
from email.mime.text import MIMEText
from email.mime.base import MIMEBase
from email import encoders
import os
import mimetypes

# Configurações do e-mail
email_address = 'login'
email_password = 'senha '

# Cria a mensagem como MIMEMultipart
msg = MIMEMultipart()
msg['Subject'] = 'assuntos'
msg['From'] = email_address
msg['To'] = 'destinatario'

# Corpo do e-mail
body = """Prezados, Thiago e Paulo,

Segue em anexo o relatório de cotações extraído e processado.

Dados extraídos e salvos com sucesso! Total de citações: 100
Número de citações: 100
Autor mais recorrente: Albert Einstein
Tag mais utilizada: love

Atenciosamente,
[VINICIUS BARBOSA]
"""
msg.attach(MIMEText(body, 'plain'))

# Anexar arquivo
arquivo = "quotes.csv"
if os.path.exists(arquivo):
    with open(arquivo, "rb") as f:
        mime_type, _ = mimetypes.guess_type(arquivo)
        mime_type = mime_type or "application/octet-stream"
        part = MIMEBase(mime_type.split("/")[0], mime_type.split("/")[1])
        part.set_payload(f.read())
        encoders.encode_base64(part)
        part.add_header('Content-Disposition', f'attachment; filename={os.path.basename(arquivo)}')
        msg.attach(part)

try:
    # Conecta ao servidor
    server = smtplib.SMTP('smtp.gmail.com', 587)
    server.starttls()  # Inicia a conexão TLS
    server.login(email_address, email_password)  # Faz login
    server.send_message(msg)  # Envia o e-mail
    print("E-mail enviado com sucesso!")
except Exception as e:
    print(f"Erro ao enviar e-mail: {e}")
finally:
    server.quit()  # Encerra a conexão


E-mail enviado com sucesso!


In [1]:
import pandas as pd

# Lê o arquivo CSV
df = pd.read_csv('quotes.csv')

# Converte o DataFrame para uma lista de dicionários
quotes_data = df.to_dict(orient='records')

# Gera o conteúdo do arquivo Python
python_code = f"""# Arquivo gerado automaticamente a partir de quotes.csv
quotes = {quotes_data}
"""

# Salva o conteúdo em um arquivo Python
with open('quotes.py', 'w', encoding='utf-8') as file:
    file.write(python_code)

print("Arquivo 'quotes.py' gerado com sucesso!")

Arquivo 'quotes.py' gerado com sucesso!


In [3]:
import smtplib
from email.mime.multipart import MIMEMultipart
from email.mime.text import MIMEText
from email.mime.base import MIMEBase
from email import encoders
import os
import mimetypes

# Configurações do e-mail
email_address = 'login'
email_password = 'senha '

# Cria a mensagem como MIMEMultipart
msg = MIMEMultipart()
msg['Subject'] = 'assunto'
msg['From'] = email_address
msg['To'] = 'destinatario'

# Corpo do e-mail
body = """Prezados, Thiago e Paulo,

 

Atenciosamente,
[VINICIUS BARBOSA]
"""
msg.attach(MIMEText(body, 'plain'))

# Anexar arquivo
arquivo = "quotes.py"
if os.path.exists(arquivo):
    with open(arquivo, "rb") as f:
        mime_type, _ = mimetypes.guess_type(arquivo)
        mime_type = mime_type or "application/octet-stream"
        part = MIMEBase(mime_type.split("/")[0], mime_type.split("/")[1])
        part.set_payload(f.read())
        encoders.encode_base64(part)
        part.add_header('Content-Disposition', f'attachment; filename={os.path.basename(arquivo)}')
        msg.attach(part)

try:
    # Conecta ao servidor
    server = smtplib.SMTP('smtp.gmail.com', 587)
    server.starttls()  # Inicia a conexão TLS
    server.login(email_address, email_password)  # Faz login
    server.send_message(msg)  # Envia o e-mail
    print("E-mail enviado com sucesso!")
except Exception as e:
    print(f"Erro ao enviar e-mail: {e}")
finally:
    server.quit()  # Encerra a conexão


E-mail enviado com sucesso!
